# Classifieur Quantique (QSVM) — Le B.A.-BA
Pas à pas : **entraîner**, **prédire**, **visualiser**.
Aucun prérequis en IA.


## 1) Installation (à exécuter une seule fois si besoin)

In [ ]:
%%bash
pip install -q qiskit qiskit-aer qiskit-machine-learning scikit-learn matplotlib

## 2) Imports et simulateur

In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
sim = AerSimulator()

## 3) Idée en deux lignes
- Un **SVM** classe grâce à un **noyau** (mesure de similarité).
- Un **QSVM** utilise un **noyau quantique** calculé par un circuit — similarité = overlap entre états.

## 4) Données jouet (moons)

In [ ]:
X, y = datasets.make_moons(n_samples=300, noise=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
plt.figure()
plt.scatter(X_train[:,0], X_train[:,1], c=y_train)
plt.title("Données d'entraînement (moons)")
plt.xlabel('x1'); plt.ylabel('x2'); plt.show()

## 5) Feature map quantique + noyau quantique

In [ ]:
num_qubits = X.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2)
qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=sim)

## 6) Entraînement du QSVM (SVC noyau pré-calculé)

In [ ]:
K_train = qkernel.evaluate(X_train, X_train)
clf = SVC(kernel='precomputed')
clf.fit(K_train, y_train)
K_test = qkernel.evaluate(X_test, X_train)
y_pred = clf.predict(K_test)
print('Accuracy test:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

## 7) Visualisation de la frontière de décision

In [ ]:
def plot_decision_boundary(model, kernel, X_train, y_train):
    x_min, x_max = X[:,0].min() - 0.5, X[:,0].max() + 0.5
    y_min, y_max = X[:,1].min() - 0.5, X[:,1].max() + 0.5
    h = 0.02
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    grid = np.c_[xx.ravel(), yy.ravel()]
    K_grid = kernel.evaluate(grid, X_train)
    Z = model.predict(K_grid).reshape(xx.shape)
    plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X_train[:,0], X_train[:,1], c=y_train, edgecolors='k')
    plt.title('Frontière de décision QSVM')
    plt.xlabel('x1'); plt.ylabel('x2'); plt.show()

plot_decision_boundary(clf, qkernel, X_train, y_train)

## 8) Prédictions sur **nouvelles données**

In [ ]:
X_new = np.array([
    [-0.5, 0.8],
    [1.5, -0.2],
    [0.2, 1.0]
])
K_new = qkernel.evaluate(X_new, X_train)
y_new = clf.predict(K_new)
print('Prédictions nouvelles données:', y_new)

## 9) Conseils pratiques
- Si c'est lent, réduis `n_samples` ou `reps` du feature map.
- Pour des résultats plus stables, fixe `random_state`.
- Si `QuantumKernel` est déprécié dans ta version, cherche `FidelityQuantumKernel` / `StateFidelity` et garde `SVC(kernel='precomputed')`.